# SCAFFOLD for Non-IID Setting 

## Introduction

In this notebook we implement the SCAFFOLD algorithm, as proposed by Karimireddy et al. in the paper [SCAFFOLD: Stochastic Controlled Averaging for Federated Learning](https://proceedings.mlr.press/v119/karimireddy20a/karimireddy20a.pdf) in 2020. The general idea is to introduce control variates to the local model updates, in order to mitigate the effects that data heterogeneity has on the learning process. 

In [2]:
from typing import List, Tuple, Optional, Dict
from pathlib import Path
from logging import WARN, INFO, DEBUG

import numpy as np
import torch
import torch.nn as nn
from torch.optim import SGD
from datasets.utils.logging import disable_progress_bar

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import (
    Metrics,
    Parameters,
    Context, 
    Code,
    ArrayRecord,
    Scalar, 
    ndarrays_to_parameters,
    parameters_to_ndarrays,
    )
from flwr.common.typing import GetParametersIns
from flwr.common.logger import log
from flwr.server import ServerApp, ServerConfig, ServerAppComponents, Server
from flwr.server.strategy import Strategy, FedAvg
from flwr.server.client_manager import ClientManager, SimpleClientManager
from flwr.server.server import FitResultsAndFailures, fit_clients
from flwr.simulation import run_simulation

Next, we need to make some imports from our own package. We will import the ```SmallCNN``` neural net object, as this is what we are running all the tests with. We will also import the function ```load_datasets```, which returns a tuple of ```(trainloader,valloader,testloader)```. We will again make use of the ```test``` function, which was introduced in the previous notebook, as well as some utility functions ```get_parameters``` and ```set_parameters```. Feel free to revisit the code, or you can simply use the python function ```help``` (eg ```help(test)```).

In [3]:
from fedlearn.model import SmallCNN, test
from fedlearn.data_loader import load_datasets
from fedlearn.utils import set_parameters, get_parameters

In [4]:
help(load_datasets)

Help on function load_datasets in module fedlearn.data_loader:

load_datasets(partition_id: int, partition_method: str, partitioner_kwargs: Dict[str, Union[str, int, float]], batch_size: int = 64, cache_dir: str = 'data', data_share_fraction: float = 0.0, data_share_seed: int = 42) -> Tuple[torch.utils.data.dataloader.DataLoader, torch.utils.data.dataloader.DataLoader, torch.utils.data.dataloader.DataLoader]
    function for loading CIFAR-10 dataset and partitioning it for federated learning.
    
    Parameters:
        partition_id:           int, the ID of the partition to load.
        partition_method:       str, the method to use for partitioning the dataset.
        partitioner_kwargs:     Dict[str, Union[str, int, float]], the parameters for the partitioner.
        batch_size:             int, the size of each batch for training and validation.
        cache_dir:              str, the directory to cache the dataset.
        data_share_fraction:    Optional[float], fraction of 

In [6]:
DATADIR = Path().cwd().parent / "data" / "flower_dataset"       # specify your data directory

DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # if running on mac, use: "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cuda
Flower 1.18.0 / PyTorch 2.7.0+cu126


We are now ready to specify some simulation parameters. We arbitrarily set the number of clients/partitions to 10, specify a batch size of 64, and choose the dirichlet method for partitioning the data, with $\alpha=0.5$. For other data partitioning schemes, we encourage you to have a look at the ```flwr_datasets``` documentation on partitioners [here](https://flower.ai/docs/datasets/ref-api/flwr_datasets.partitioner.html).

In [7]:
NUM_PARTITIONS = 10 # Number of partitions for the federated dataset same as the number of clients
BATCH_SIZE = 32
PARTITION_METHOD = "dirichlet"  # Options: "iid", "dirichlet", "shard"
PARTITIONER_KWARGS = {
    "num_partitions": NUM_PARTITIONS,   # Number of partitions to create
    "alpha": 0.5,                       # Dirichlet parameter, only used if partition_method is "dirichlet"
    "partition_by": "label"             # Partition by label, only used if partition_method is "dirichlet" or "shard"
}

# Load datasets for partition 0 to check if everything works
_, _, _ = load_datasets(
    partition_id=0,                         # specify partition ID to load
    partition_method=PARTITION_METHOD,      # Method to partition the dataset
    partitioner_kwargs=PARTITIONER_KWARGS,  # Parameters for the partitioner
    batch_size=BATCH_SIZE,                  # Batch size for the DataLoader    
    cache_dir=DATADIR                       # Directory to cache the datasets
)

### Define Scaffold Optimizer

Recall that the local update in Scaffold is given by

$$
w^{(i)} \gets w^{(i)} - \eta_l \left( g_i(w^{(i)}) + c - c_i \right)
$$

Which can be seen as a gradient correction to Stochastic Gradient Descent (SGD). We may therefore extend the pytorch ```SGD``` class. We do this by computing the the regular SGD step, then adding the correction manually:

$$
\begin{align*}
w^{(i)} &\gets w^{(i)} - \eta_l \, g_i\left(w^{(i)}\right) \\
w^{(i)} &\gets w^{(i)} - \eta_l (c - c_i)
\end{align*}
$$

In [8]:
class ScaffoldOptimizer(SGD):
    """
    Extension of the SGD optimizer for the Scaffold algorithm.
    This optimizer applies a correction term based on the global 
    and client control variables. 
    """
    def __init__(self, 
        params, 
        lr: float, 
        momentum: float = 0., 
        weight_decay: float = 0.
    ) -> None: 
        super().__init__(params, lr, momentum, weight_decay)

    def step_custom(
        self, 
        global_cv: List[torch.Tensor], 
        client_cv: List[torch.Tensor]
    ) -> None:
        """
        Perform a single optimization step.
        :param global_cv: Global control variable
        :param client_cv: Client control variable
        """
        # compute regular SGD step
        #   w <- w - lr * grad
        super().step() 

        # now add the correction term
        #   w <- w - lr * (g_cv - c_cv)
        device = self.param_groups[0]["params"][0].device
        for group in self.param_groups:
            for param, g_cv, c_cv in zip(group["params"], global_cv, client_cv):
                # here we add the correction term to each parameter tensor.
                # the alpha value scales the correction term
                    g_cv, c_cv = g_cv.to(device), c_cv.to(device)
                    param.data.add_(g_cv - c_cv, alpha=-group["lr"]) 

As in the previous notebook, we need to implement a ```train``` function that performs the local update for a client. As the control variates are needed for the correction step, we cannot simply reuse the function from the previous notebook. Therefore, we need to implement a similar function, that takes the control variates into account.

In [22]:
def train_scaffold(
    net: torch.nn.Module, 
    device: torch.device, 
    trainloader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    num_epochs: int, 
    lr: float, 
    momentum: float, 
    weight_decay: float, 
    global_cv: List[torch.Tensor], 
    client_cv: List[torch.Tensor],
) -> None:
    """
    Function that trains a model using the Scaffold optimization algorithm.
    Parameters:
        net:            The neural network model to train.
        device:         The device to run the training on (CPU or GPU).
        trainloader:    DataLoader for the training data.
        criterion:      Loss function to use for training.
        num_epochs:     Number of epochs to train the model.
        lr:             Learning rate for the optimizer.
        momentum:       Momentum factor for the optimizer.
        weight_decay:   Weight decay (L2 penalty) for the optimizer.
        global_cv:      Global control variables for Scaffold.
        client_cv:      Client control variables for Scaffold.
    """
    net.to(device)
    net.train()

    # initialize the custom Scaffold optimizer
    optimizer = ScaffoldOptimizer(
        net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay
    )
    
    for _ in range(num_epochs):
        for i, batch in enumerate(trainloader):
            Xtrain, Ytrain = batch["img"].to(device), batch["label"].to(device)
            optimizer.zero_grad()
            output = net(Xtrain)
            loss = criterion(output, Ytrain)

            loss.backward()
            
            # Perform a single optimization step with the control variables
            optimizer.step_custom(global_cv, client_cv)

Now we get to the difficult part. As is the previous notebook, we need to define a custom client class for the flower framework. However, as the algorithm requires aggregation and storage of a global control variate as well, we must also implement both the aggregation class, called a ```Strategy```, as well as the server class, called a ```Server```, from scratch.

We will start with the client class, as we are most familiar with this. Below are some important considerations:

1. We can inherit the  ```NumPyClient``` class from the flower framework, however we need to remember to convert between ```np.ndarray``` and ```torch.tensor``` before and after local updates.
2. There are 3 methods we need to implement, with a predefined structure
   1. ```fit()```
   2. ```get_parameters()```
   3. ```evaluate()```

The ```fit()``` method takes 2 parameters, ```parameters```, which in our case will be the concatenated global model parameters and the global control variate, and ```config```, which we will ignore for now. This method is responsible for all client-side updates, which means that we will call the ```train_scaffold``` function inside it to perform the local model update, as well as computing the updated local control variate. We will update the local control variate according to Option II from the original paper, which is given by:

$$
c_i^+ \gets c_i - c + \frac{1}{K \eta_l}(w_t - w_{t+1}^{(i)}) \tag{1}
$$

Where $K$ is the number of of descent steps taken, ie number of epochs times the number of mini-batches. 

We also need to specify what exactly is returned by the method. For clarity and in order to remain close in notation to the original paper, we will return the difference between the input model parameters and the locally updated parameters (returned from ```train_scaffold```), concatenated with the difference between the new and previous local control variate. Concretely, we return

$$
[w_{t+1}^{(i)} - w_t,\,\, c_i^+ - c_i]
$$

This is due to how the server-side updates look:

$$
\begin{align*}
   w_{t+1} &\gets w_t + \frac{\eta_g}{|\mathcal{S}|} \sum_{i \in \mathcal{S}} \frac{n^{(i)}}{n}(w_{t+1}^{(i)} - w_t) \tag{2.1} \\
   c &\gets c + \frac{1}{N} \sum_{i \in \mathcal{S}} \frac{n^{(i)}}{n}(c_i^+ - c_i) \tag{2.2}
\end{align*}
$$

In [23]:
aa = [np.zeros((i, i)) for i in range(1, 4)] 

bb = [torch.zeros(a.shape) for a in aa]

print(bb)

[tensor([[0.]]), tensor([[0., 0.],
        [0., 0.]]), tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])]


In [24]:
class ScaffoldClient(NumPyClient):
    def __init__(
        self, 
        partition_id: int, 
        net: torch.nn.Module, 
        trainloader: torch.utils.data.DataLoader, 
        valloader: torch.utils.data.DataLoader,
        criterion: nn.Module,
        device: torch.device,
        num_epochs: int,
        lr: float,
        momentum: float,
        weight_decay: float,
        context: Context,
    ) -> None:
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.criterion = criterion
        self.device = device
        self.num_epochs = num_epochs
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay

        self.client_state = context.state.array_records
        self.client_cv_header = "client_cv"

        # define directory to save client control variates
        ##if save_dir is None:
        ##    save_dir = "client_cvs"

        # create directory if it does not exist
        ##if not os.path.exists(save_dir):
        ##    os.makedirs(save_dir)

        # define the path to save the client control variates
        ##self.save_name = os.path.join(save_dir, f"client_{self.partition_id}_cv.pt")

        # initialize client control variates
        ##self.client_cv = [torch.zeros(param.shape).to(torch.float32) for param in self.net.state_dict().values()]


    # Here is where all the training logic and control variate updates happen
    def fit(self, parameters: List[np.ndarray], config: dict) -> Tuple[List[np.ndarray], int, dict]:

        # the global parameters are packed together with the global control variates
        # in the form [params, global_cv]. we start by separating them
        params = parameters[:len(parameters) // 2]          # list of np.ndarray
        global_cv = parameters[len(parameters) // 2:]       # list of np.ndarray

        # load the current global model:
        set_parameters(self.net, params)

        # load client control variates, if they exist:
        if self.client_cv_header in self.client_state:
            client_cv = self.client_state[self.client_cv_header].to_numpy_ndarrays() # list of np.ndarray
        else:
            # if no client control variates exist, initialize them to zero arrays
            client_cv = [np.zeros_like(p) for p in params]  # list of np.ndarray

        client_cv_torch = [torch.tensor(cv).to(torch.float32) for cv in client_cv]  # list of torch.tensor

        #if os.path.exists(self.save_name):
        #    self.client_cv = torch.load(self.save_name)     # list of torch.tensor

        # convert global control variates to tensors
        global_cv_torch = [torch.tensor(cv).to(torch.float32) for cv in global_cv]  # list of torch.tensor

        # call the training function
        train_scaffold(
            net=self.net,
            device=self.device,
            trainloader=self.trainloader,
            criterion=self.criterion,
            num_epochs=self.num_epochs,
            lr=self.lr,
            momentum=self.momentum,
            weight_decay=self.weight_decay,
            global_cv=global_cv_torch,           # passing list of torch.tensor
            client_cv=client_cv_torch            # passing list of torch.tensor
        )

        # update the client control variates
        yi = get_parameters(self.net)           # list of np.ndarray

        # compute coefficient for the control variates
        # 1 / (K * eta) where K is the number of backward passes (num_epochs * len(trainloader))
        coeff = 1. / (self.num_epochs * len(self.trainloader) * self.lr) 

        # define new list for udated client control variates
        client_cv_new = []

        # compute client control variate update according to eq (1), list of np.ndarray
        for xj, yj, cj, cij in zip(params, yi, global_cv, client_cv):
            client_cv_new.append(
                cij - cj + coeff * (xj - yj)
            ) 

        # compute server updates
        server_update_x = [yj - xj for xj, yj in zip(params, yi)]
        server_update_c = [cij_n - cij for cij_n, cij in zip(client_cv_new, client_cv)]

        # convert client cvs back to torch tensors
        self.client_state[self.client_cv_header] = ArrayRecord(client_cv_new)

        ##self.client_cv = [torch.tensor(cv).to(torch.float32) for cv in client_cv_new]  

        # save the updated client control variates
        ##torch.save(self.client_cv, self.save_name)

        #concatenate server updates
        server_update = server_update_x + server_update_c

        return server_update, len(self.trainloader.dataset), {}


    def get_parameters(self, config: dict) -> List[np.ndarray]:
        return get_parameters(self.net)


    def evaluate(self, parameters: List[np.ndarray], config: dict) -> Tuple[float, int, dict]:
        set_parameters(self.net, parameters)
        avg_loss, accuracy = test(
            net=self.net,
            device=self.device,
            testloader=self.valloader,
            criterion=self.criterion
        )
        return float(avg_loss), len(self.valloader), {"accuracy": accuracy}

Now that we have the flower client defined, we need to define a constructor function which the flower framework can use to instatiate clients as it goes. This is done similarly to in the previous notebook. It should be noted that all the simulation parameters, such as ```partition_method```, ```batch_size``` etc. can be passed to the ```client_fn``` function thru the context. This is useful when simulations are run from the command line. For now, we will ignore this, and hardcode it into the function.

In [25]:
def client_fn(context: Context) -> Client:
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    partitioner_kwargs = {
        "num_partitions": num_partitions,   # Number of partitions to create
        "alpha": 0.5,                       # Dirichlet parameter
        "partition_by": "label"             # Partition by label
    }

    trainloader, valloader, _ = load_datasets(
        partition_id=partition_id,              # specify partition ID to load
        partition_method=PARTITION_METHOD,      # Method to partition the dataset
        partitioner_kwargs=partitioner_kwargs,  # Parameters for the partitioner
        batch_size=BATCH_SIZE,                  # Batch size for the DataLoader    
        cache_dir=DATADIR                       # Directory to cache the datasets
    )

    net = SmallCNN().to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    
    # Define hyperparameters for training
    num_epochs = 5
    lr = 0.01
    momentum = 0.5
    weight_decay = 0.

    return ScaffoldClient(
        partition_id=partition_id,
        net=net,
        trainloader=trainloader,
        valloader=valloader,
        criterion=criterion,
        device=DEVICE,
        num_epochs=num_epochs,
        lr=lr,
        momentum=momentum,
        weight_decay=weight_decay,
        context=context,
        #save_dir=LOGDIR / "client_cvs"
    ).to_client()


client = ClientApp(client_fn=client_fn)

Finally, we are ready to define the server. The ```Server``` class is responsible for many things, however the main ones we are concerned with are how the initial parameters are initialised, and how the global updates are performed.

We define a ```_get_initial_parameters()``` method that is essentially the same as from the base ```Server``` class, however we need to also initialise the global control variate.

The ```fit_round()``` method is responsible for computing the global updates. This contains all the logic for running a single round of federated learning. The things that occur are:
1. Send the current global model parameters and global control variate to the selected clients
2. Collect the results/failures from the clients
3. Aggregate the results
4. Update the global model parameters and global control variate according to equations 2.1, 2.2

In [26]:
def concat_params(
    parameters: Parameters, 
    global_cv: List[np.ndarray]
) -> Parameters:
    """
    Concatenate model parameters and global control variates.
    """
    parameters_ndarrays = parameters_to_ndarrays(parameters)
    parameters_ndarrays.extend(global_cv)
    return ndarrays_to_parameters(parameters_ndarrays)


class ScaffoldServer(Server):

    def __init__(
        self, 
        strategy: Strategy, 
        client_manager: Optional[ClientManager] = None,
    ) -> None:
        
        if client_manager is None:
            client_manager = SimpleClientManager()

        super().__init__(strategy=strategy, client_manager=client_manager)
        
        self.global_cv: List[np.ndarray] = []  # Global control variates for Scaffold
    
    def _get_initial_parameters(
        self, 
        server_round: int, 
        timeout: Optional[float]
    ) -> Parameters: 
        
        parameters: Optional[Parameters] = self.strategy.initialize_parameters(
            self.client_manager
        )
    
        if parameters is not None:
            log(INFO, "Using initial parameters provided by strategy")
        else:
            # Get initial parameters from one of the clients
            log(INFO, "Requesting initial parameters from one random client")
            random_client = self._client_manager.sample(1)[0]
            ins = GetParametersIns(config={})
            get_parameters_res = random_client.get_parameters(
                ins=ins, timeout=timeout, group_id=server_round
            )
            if get_parameters_res.status.code == Code.OK:
                log(INFO, "Received initial parameters from one random client")
            else:
                log(
                    WARN,
                    "Failed to receive initial parameters from the client."
                    " Empty initial parameters will be used.",
                )
            parameters = get_parameters_res.parameters
            
        self.global_cv = [
                np.zeros_like(param, dtype=np.float32) for param in parameters_to_ndarrays(parameters)
            ]
        
        return parameters


    def fit_round(
        self,
        server_round: int,
        timeout: Optional[float],
    ) -> Optional[Tuple[Optional[Parameters], Dict[str, Scalar], FitResultsAndFailures]]:
        
        # define client instructions to be passed to "fit_clients" function
        client_instructions = self.strategy.configure_fit(
            server_round=server_round,
            parameters=concat_params(self.parameters, self.global_cv),  # send both model parameters and global control variates
            client_manager=self._client_manager,
        )

        # if no clients are selected, return None
        if not client_instructions:
            log(INFO, f"fit_round {server_round}: no clients selected.")
            return None
        
        log(
            DEBUG,
            f"fit_round {server_round}: selected {len(client_instructions)} clients.",
        )

        # Call the "fit_clients" function to perform the training on selected clients
        results, failures = fit_clients(
            client_instructions=client_instructions,
            max_workers=self.max_workers,
            timeout=timeout,
            group_id=server_round,
        )

        log(
            DEBUG,
            f"fit_round {server_round}: received {len(results)} results and {len(failures)} failures.",
        )

        # Aggregate the results from the clients
        aggregated_results = self.strategy.aggregate_fit(
            server_round=server_round,
            results=results,
            failures=failures,
        )

        # Extract the aggregated parameters and control variates
        aggregated_results_combined = []
        if aggregated_results[0] is not None:
            aggregated_results_combined = parameters_to_ndarrays(aggregated_results[0])

        # Split the aggregated results into model parameters and control variates
        aggregated_parameters = aggregated_results_combined[:len(aggregated_results_combined) // 2] # model parameters
        aggregated_cv = aggregated_results_combined[len(aggregated_results_combined) // 2:]         # control variates

        # define the update coefficient for the control variates
        cv_coeff = len(results) / len(self._client_manager.all())

        # Update the global control variates according to
        # global_cv <- global_cv + cv_coeff * aggregated_cv
        # where cv_coeff = |S| / N, |S| is the number of clients that participated in the round
        # and aggregated_cv = (1 / |S|) * sum_{i in S} (c_i^+ - c_i)
        self.global_cv = [
            cv + cv_coeff * new_cv for cv, new_cv in zip(self.global_cv, aggregated_cv)
        ]


        # Update the global model parameters
        # new_parameters = current_parameters + aggregated_parameters
        # where current_parameters are the parameters of the global model before the round
        # and aggregated_parameters = (1 / |S|) * sum_{i in S} (w_i^+ - w)
        current_parameters = parameters_to_ndarrays(self.parameters)
        new_parameters = [
            param + update for param, update in zip(current_parameters, aggregated_parameters)
        ]

        new_parameters = ndarrays_to_parameters(new_parameters)

        return new_parameters, aggregated_results[1], (results, failures)

Again we need to create a server_fn, just like in the previous notebook.

In [27]:
# Create an instance of the model and get the parameters
params = get_parameters(SmallCNN())
criterion = nn.CrossEntropyLoss()

# The `evaluate` function will be called by Flower after every round
def evaluate(
    server_round: int,
    parameters: list[np.ndarray],
    config: dict[str, Scalar],
) -> Optional[Tuple[float, dict[str, Scalar]]]:
    
    net = SmallCNN().to(DEVICE)
    _, _, testloader = load_datasets(   
        partition_id=0,                         # specify partition ID to load
        partition_method=PARTITION_METHOD,      # Method to partition the dataset
        partitioner_kwargs=PARTITIONER_KWARGS,  # Parameters for the partitioner
        batch_size=BATCH_SIZE,                  # Batch size for the DataLoader    
        cache_dir=DATADIR                       # Directory to cache the datasets
    )
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(
        net=net, 
        device=DEVICE, 
        testloader=testloader, 
        criterion=criterion,
        )
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}


def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,                                   # Use all clients for training, C
        fraction_evaluate=0.5,                              # Use 50% of clients for evaluation
        min_fit_clients=10,                                 # Minimum number of clients to train
        min_evaluate_clients=5,                             # Minimum number of clients to evaluate
        min_available_clients=NUM_PARTITIONS,               # Minimum number of clients available (enforce all clients to be available)
        evaluate_fn=evaluate,                               # Pass the evaluation function
        initial_parameters=ndarrays_to_parameters(params),  # Initial model parameters
        evaluate_metrics_aggregation_fn=weighted_average,   # Custom aggregation function for evaluate metrics
    )

    server = ScaffoldServer(strategy=strategy)

    # Configure the server for desired number of rounds
    config = ServerConfig(num_rounds=2)
    return ServerAppComponents(server=server, config=config)

# Create the ServerApp
server = ServerApp(server_fn=server_fn)

In [ ]:
backend_config = {
    #"ray_init_args": {
    #    "num_cpus": 1,
    #    "num_gpus": 1,
    #},
    "client_resources": {
        "num_cpus": 2,
        "num_gpus": 0.2,
    }
}
run_simulation(
    server_app=server, client_app=client, num_supernodes=NUM_PARTITIONS, backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      


INFO :      [INIT]
INFO :      Using initial parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 2.3021192284056933, {'accuracy': 0.099}
INFO :      
INFO :      [ROUND 1]


Server-side evaluation loss 2.3021192284056933 / accuracy 0.099


INFO :      fit progress: (1, 2.006978388030689, {'accuracy': 0.2481}, 87.7937350999564)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Server-side evaluation loss 2.006978388030689 / accuracy 0.2481


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      fit progress: (2, 1.6040264360440044, {'accuracy': 0.4086}, 171.48757829982787)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Server-side evaluation loss 1.6040264360440044 / accuracy 0.4086


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 172.91s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.9975258550011967
INFO :      		round 2: 1.6646488485096385
INFO :      	History (loss, centralized):
INFO :      		round 0: 2.3021192284056933
INFO :      		round 1: 2.006978388030689
INFO :      		round 2: 1.6040264360440044
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.2255864589381928), (2, 0.3523500286068103)]}
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.099), (1, 0.2481), (2, 0.4086)]}
INFO :      
